In [56]:
get_ipython().magic('matplotlib inline')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from forward_selection import forward_selected
from backward_selection import backward_selected
sns.set_style('white')
sns.set_context('notebook')

data=pd.read_excel('caschool.xlsx.xls')

state=440232650+470353886+470352982 # sum of the student IDs for the members of the group

train = data.sample(frac=0.8, random_state=state) # For tasks 1-5 
test = data[data.index.isin(train.index)==False].copy() # Only for prediction (task 6)

train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [57]:
# Matt preps the data for the models
cols = list(train.columns.values)

for s in [u'enrl_tot', u'teachers', u'calw_pct', u'meal_pct', u'computer', u'testscr', u'comp_stu', u'expn_stu', u'str', u'avginc', u'el_pct', u'read_scr', u'math_scr']:
    train["log_" + s.encode('utf-8')] = train[s].apply(np.log)
    train["sqrt_" + s.encode('utf-8')] = train[s]**0.5
    
# Take computer as an interaction variable. 
# Round comp_stu to the nearest 0.1 with:
train['round_comp_stu'] = train['comp_stu'].round(1)

# Take gr_span as dummy vars:
train["KK-08"] = np.where(train["gr_span"]=="KK-08",1,0)
train["KK-06"] = np.where(train["gr_span"]=="KK-06",1,0)

In [58]:
train.head()

,Observation Number,dist_cod,county,district,gr_span,enrl_tot,teachers,calw_pct,meal_pct,computer,...,sqrt_avginc,log_el_pct,sqrt_el_pct,log_read_scr,sqrt_read_scr,log_math_scr,sqrt_math_scr,round_comp_stu,KK-08,KK-06
0,357,70995,Sonoma,Waugh Elementary,KK-06,734,36.000000,2.0243,12.820500,117,...,4.117766,1.877862,2.557246,6.516341,26.001923,6.506232,25.870833,0.2,0,1
1,15,72298,Tulare,Woodville Elementary,KK-08,649,36.000000,14.6379,76.271202,31,...,3.103224,4.383566,8.951160,6.416569,24.736613,6.431331,24.919872,0.0,1,0
2,60,72561,Ventura,Rio Elementary,KK-08,3074,142.550003,11.2898,66.194901,249,...,3.404703,3.641755,6.177277,6.450786,25.163466,6.449680,25.149553,0.1,1,0
3,259,72207,Tulare,Three Rivers Union Elementary,KK-08,248,11.120000,7.4627,21.268700,51,...,3.919821,-inf,0.000000,6.503989,25.841826,6.481271,25.549951,0.2,1,0
4,246,63024,Humboldt,Scotia Union Elementary,KK-08,370,19.799999,6.5041,36.314400,56,...,3.489126,-0.615186,0.735215,6.498282,25.768197,6.481118,25.547994,0.2,1,0


For each of the proposed models we will want: 
1. OLS
2. VIF
3. SER
4. $R^2-adj$ 
5. Residual Plot

In [ ]:
# creating a log of a variable/predictor:
train['log_testscr or variable'] = np.log(train['testscr or variable'])

# to square-root Y:
train['testscr_sqr'] = np.sqrt(train['testscr'])

# to square Y:
train['testscr_sq2'] = train['testscr']**2

# creating knots for linear splines - this example splits at 20,40,60 and 80%:
xi1=train['testscr'].quantile(.2) 
xi2=train['testscr'].quantile(.4)
xi3=train['testscr'].quantile(.6)
xi4=train['testscr'].quantile(.8)
train['Step1']=(train['testscr']>xi1)*(train['testscr']-xi1)
train['Step2']=(train['testscr']>xi2)*(train['testscr']-xi2)
train['Step3']=(train['testscr']>xi3)*(train['testscr']-xi3)
train['Step4']=(train['testscr']>xi4)*(train['testscr']-xi4)

# creating cubic splines using knot positions defined above
train['Step31']=(train['testscr']>xi1)*(train['testscr']-xi1)**3
train['Step32']=(train['testscr']>xi2)*(train['testscr']-xi2)**3
train['Step33']=(train['testscr']>xi3)*(train['testscr']-xi3)**3
train['Step34']=(train['testscr']>xi4)*(train['testscr']-xi4)**3

In [ ]:
# 1. OLS
import statsmodels.formula.api as smf
import statsmodels.api as sm
# insert formula you want, str must always be included as a variable

# for interation effects use 'C:(variable1, variable2)'

# to create a polynomial use '+ np.power(EngDispl, 2)' where 2 is the degree you're raising the variable to 
formula='testscr ~ str + avginc + el_pct + expn_stu + comp_stu'
ols1 = smf.ols(formula=formula, data=train).fit()
resid1 = ols1.resid
fitted1 = ols1.fittedvalues
ols1.summary()

In [ ]:
# 2. VIF

features = train[['str','avginc','el_pct', 'expn_stu','comp_stu']] # add in all varibles for current model
features = sm.add_constant(features)  # make sure to include a column of 1s when using the variance inflation factor function.

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = []
for i in range(6): # range is number of selected variables + 1
    vif.append(variance_inflation_factor(features.values, i+1))
    
print(vif)

In [ ]:
# average VIF
np.mean(vif)

In [ ]:
# 3. and 4.  SER, Rsq and Rsq adj

# For a linear Y:
ols1.mse_resid**0.5 # SER

    # Rsq and Rsq Adj are in the OLS summary

# For logY:
    # can do this without bias correction (BC), Duan BC (non-normal errors), or Normal BC (normal errors)
    # this example assumes a loglin model, its the same with a loglog model

eres1 = np.exp(resid1) # exponential of errors
n = len(resid1)
fp1 = sum(eres1)/n # Duan BC factor
fp11 = np.exp(ols1.mse_resid/2) # Normal BC factor

    # untransform Y 
testscr_loglin = np.exp(fitted1) # no BC
testscr_loglin1 = np.exp(fitted1) * fp1 # Duan
testscr_loglin11 = np.exp(fitted1) * fp11 # Normal

    # get variance for calculating SER, R2 etc, it will never change
stats.describe(train['testscr'])

    # calculate new residuals and then SER, Rsq and Rsq adj for each BC, p is the number of predictors
res_loglin = train['testscr']-testscr_loglin
np.sqrt(sum(res_loglin**2)/(n-p-1)), 1 - sum(res_loglin**2)/((n-1)*variance), 1 - sum(res_loglin**2)/(n-p-1)/variance

res_loglin1 = train['testscr']-testscr_loglin1
np.sqrt(sum(res_loglin1**2)/(n-p-1)), 1 - sum(res_loglin1**2)/((n-1)*variance), 1 - sum(res_loglin1**2)/(n-p-1)/variance

res_loglin11 = train['testscr']-testscr_loglin11
np.sqrt(sum(res_loglin11**2)/(n-p-1)), 1 - sum(res_loglin11**2)/((n-1)*variance), 1 - sum(res_loglin11**2)/(n-p-1)/variance


# For square-root Y
testscr_pred_sq2 = fitted1**2 #square the fitted values to get Y

    # find residuals then SER, R2, R2 adj, p is the number of predictors
res_pred_sq2 = train['testscr']-testscr_pred_sq2
np.sqrt(sum(res_pred_sq2**2)/(n-p-1)), 1 - sum(res_pred_sq2**2)/((n-1)*variance), 1 - sum(res_pred_sq2**2)/(n-p-1)/variance

# for Y^2 
testscr_pred_sqr = np.sqrt(fitted1) #square root the fitted values to get Y
    # find residuals then SER, R2, R2 adj
res_pred_sqr = train['testscr']-testscr_pred_sqr
np.sqrt(sum(res_pred_sqr**2)/(n-p-1)), 1 - sum(res_pred_sqr**2)/((n-1)*variance), 1 - sum(res_pred_sqr**2)/(n-p-1)/variance

In [ ]:
# 5. Residuals, gives LOESS, true line

fig, ax= plt.subplots()
sns.regplot(fitted1, resid1, lowess=True, ax=ax, scatter_kws={'s': 35, 'alpha': .6}) # remember to use untransformed fitted values if you transformed Y
ax.set_xlabel('Fitted',  {'fontsize': 12})
ax.set_ylabel('Residuals', {'fontsize': 12})
ax.set_title('Residuals vs Fitted - Model X') # title = current model
plt.axhline(color='Black', alpha=0.3, linestyle='--')  
plt.show()

In [ ]:
# Histogram of residuals (to check whether the distrubution is uniform/normal)

ax = sns.distplot(resid1, bins=30)
ax.set_xlabel('Residuals',  {'fontsize': 12})
ax.set_ylabel('Frequency', {'fontsize': 12})
ax.set_title('Risdual Histogram - Model X')
plt.show()

In [ ]:
# numerical summary of residuals
from scipy import stats
stats.describe(resid1)

In [ ]:
# gives a graph of the residuals squared with a true fit line (good for checking homoskedasticity)
tableau=['#1F77B4', '#FF7F0E', '#2CA02C', '#DB2728', '#9467BD', '#8c564b', '#e377c2', '#7f7f7f']
fig, ax= plt.subplots()
sns.regplot(fitted1,resid1**2, ci=None, fit_reg=False, scatter_kws={'s': 35, 'color': tableau[3], 'alpha': 0.7})
ax.set_xlabel('Fitted values',  {'fontsize': 12})
ax.set_ylabel('Residual squared', {'fontsize': 12})
z1 = lowess(resid**2, fitted1, frac=1./10)
plt.plot(z1[:,0],z1[:,1],'blue')
plt.show()